# Mission to Mars

In [51]:
# Import dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import os
import pymongo

In [52]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [53]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News Site
##### - Scrape the NASA Mars News Site 
##### - Collect the latest News Title and Paragraph Text
##### - Assign the text to variables that you can reference later

In [54]:
# Visit URL
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=\
publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [55]:
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

In [56]:
# Use Beautiful Soup's find() method to navigate and retrieve attributes
news_title = soup.find('div', class_="content_title").find('a').text
news_p = soup.find('div', class_="article_teaser_body").text
print(news_title)
print("-------------------------------")
print(news_p)

A Rover Pit Stop at JPL
-------------------------------
Working like a finely honed machine, a team of engineers in this time-lapse video clip install test wheels on another finely honed machine: NASA's Mars 2020 rover.


### Mars Space Images - Featured Image
##### - Use splinter to navigate the site and find the image url for the current Featured Mars Image 
##### - Assign the url string to a variable called featured_image_url
##### - Make sure to find the image url to the full size .jpg image
##### - Make sure to save a complete url string for this image

In [57]:
# Visit URL
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)
# Use splinter to click to get to the full size image
browser.click_link_by_partial_text('FULL IMAGE')

In [58]:
# JPL HTML object
jpl_html = browser.html
# Parse with bs
jpl_soup = bs(jpl_html, 'html.parser')

In [59]:
# Use BS's find() method to navigate and retrieve attributes
featured_image = jpl_soup.find('img', class_="fancybox-image")['src']
# Store featured image URL as f string
featured_image_url = f'https://www.jpl.nasa.gov{featured_image}'
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16021_ip.jpg


### Mars Weather
##### - Visit the Mars Weather twitter account, scrape the latest Mars weather tweet 
##### - Save the tweet text for the weather report as a variable called 'mars_weather'

In [60]:
twitter_url = 'https://twitter.com/marswxreport?lang=en'
# Retrieve page with the requests module
twitter_response = requests.get(twitter_url)
# Create BS object; parse with html.parser
twitter_soup = bs(twitter_response.text, 'html.parser')
# Use BS's find() method to navigate and retrieve attributes
twitter_result = twitter_soup.find('div', class_="js-tweet-text-container")
# Use BS's find() method to retrieve text of tweet
mars_weather = twitter_result.find('p', class_="js-tweet-text").text
print(mars_weather)

The #Mars2020 rover gets her wheels from the pit crew with PhDs
https://mars.nasa.gov/mars2020/mission/where-is-the-rover/ …pic.twitter.com/DRHHZpRI3f


### Mars Facts
##### - Visit the Mars Facts webpage
##### - Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
##### - Use Pandas to convert the data to a HTML table string

In [61]:
facts_url = 'https://space-facts.com/mars/'
# Use Pandas read_html function to scrape for facts table
facts_table = pd.read_html(facts_url)
# print(facts_table)
# print(facts_table) is pulling two tables, use index [1] to only return Mars facts
mars_facts_table = facts_table[1]
print(mars_facts_table)

                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers


In [62]:
# Pandas dataframe
facts_df = mars_facts_table
# Create column names
facts_df.columns = ['Description', 'Value']
facts_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [63]:
# Reset index to Description
mars_facts_df = facts_df.set_index('Description')
mars_facts_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [64]:
# Use Pandas to_html to convert to a HTML table string
mars_facts_html = mars_facts_df.to_html()
print(mars_facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Value</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


### Mars Hemispheres
##### - Visit the USGS Astrogeology site to obtain high resolution images for each of Mars' hemispheres
##### - You will need to click each of the links to the hemispheres to find the image url to the full resolution image
##### - Save both the image url string for the full resolution hemisphere image and the Hemisphere title containing the hemisphere name 
##### - Use a Python dictionary to store the data using the keys img_url and title.
##### - Append the dictionary with the image url string and the hemisphere title to a list 
##### - This list will contain one dictionary for each hemisphere

In [65]:
# Visit URL
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemi_url)
# Hemi HTML object
hemi_html = browser.html
# Parse HTML with BS
hemi_soup = bs(hemi_html, 'html.parser')

In [66]:
# Create a list to store dictionaries
hemi_list = []
# Use bs to find_all image titles to click
image_titles = hemi_soup.find_all('h3')
# Loop through each title, click to find full image, store as dictionaries, append to list
for image_title in image_titles:
    # store title text as variable
    link = image_title.text
    # use splinter to click the title's link
    browser.click_link_by_partial_text(link)
    # create second html browser for next page
    hemi_html_2 = browser.html
    # create second bs parser for next page
    hemi_soup_2 = bs(hemi_html_2, 'html.parser')
    # find image attributes for url ending
    partial_img_url = hemi_soup_2.find('img', class_="wide-image")['src']
    # store image url as f string
    img_url = f'https://astrogeology.usgs.gov{partial_img_url}'
    # store result in a dictionary
    hemi_dict = {"title": link,
                "img_url": img_url}
    # append dictionary to the list
    hemi_list.append(hemi_dict)
    # use browser's back function to go back to home page to click next title
    browser.back()
    
print(hemi_list)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]
